# Esteira de Aprendizado de Máquina

Este notebook implementa uma esteira básica de aprendizado de máquina para o conjunto de dados Heart Disease, cobrindo desde o carregamento e pré-processamento dos dados até o treinamento, avaliação e predição de um modelo.

In [ ]:
# Instalar bibliotecas necessárias (se não estiverem instaladas)
# Execute o comando abaixo no terminal ou descomente e execute aqui se necessário
# !pip install ucimlrepo pandas scikit-learn matplotlib seaborn numpy

In [ ]:
# Importar bibliotecas necessárias
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Utilizar o conjunto de dados Heart Disease

In [ ]:
# Carregar o dataset Heart Disease
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets

# Combinar features e target em um único DataFrame
df = X.copy()
df['target'] = y

print("Primeiras 5 linhas do dataset Heart Disease:")
print(df.head())

## 2. Apresentar as estatísticas descritivas gerais do conjunto de dados

In [ ]:
print("Estatísticas Descritivas:")
print(df.describe())

print("\nInformações do Dataset (tipos de dados, valores nulos):")
df.info()

# Tratar valores ausentes (se houver) com a mediana para colunas numéricas
df.fillna(df.median(numeric_only=True), inplace=True)

## 3. Aplicar ao menos uma transformação no conjunto de dados em qualquer das colunas

In [ ]:
# Criar uma nova feature 'chol_age_ratio' (relação colesterol/idade)
df['chol_age_ratio'] = df['chol'] / df['age']

print("Dataset após adição da coluna 'chol_age_ratio':")
print(df.head())

## 4. Aplicar ao menos uma transformação no conjunto de dados em qualquer das linhas

In [ ]:
# Remover outliers com base na coluna 'chol_age_ratio' (abaixo do 5º percentil)
q05_chol_age_ratio = df['chol_age_ratio'].quantile(0.05)
df_transformed = df[df['chol_age_ratio'] >= q05_chol_age_ratio]

print(f"Dataset original tinha {len(df)} linhas.")
print(f"Dataset transformado tem {len(df_transformed)} linhas após remoção de outliers (chol_age_ratio < {q05_chol_age_ratio:.2f}).")
print("Primeiras 5 linhas do dataset após transformação de linha:")
print(df_transformed.head())

## 5. Criar três subconjuntos: Treinamento, Validação e Testes

In [ ]:
# Separar features (X) e target (y)
X = df_transformed.drop('target', axis=1)
y = df_transformed['target']

# Dividir em treinamento (70%) e conjunto temporário (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Dividir conjunto temporário em validação (15%) e testes (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Shape do subconjunto de treinamento (X_train): {X_train.shape}")
print(f"Shape do subconjunto de validação (X_val): {X_val.shape}")
print(f"Shape do subconjunto de testes (X_test): {X_test.shape}")

# Escalonar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Converter arrays escalonados de volta para DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nDados escalonados (primeiras 5 linhas do treino escalonado):")
print(X_train_scaled.head())

## 6. Realizar o treinamento e a avaliação de um modelo

In [ ]:
# Inicializar o modelo (Decision Tree Classifier)
model = DecisionTreeClassifier(random_state=42)

print("Treinando o modelo...")
model.fit(X_train_scaled, y_train)
print("Modelo treinado com sucesso!")

# Avaliar o modelo no conjunto de validação
y_pred_val = model.predict(X_val_scaled)
accuracy_val = accuracy_score(y_val, y_pred_val)

print(f"\nAcurácia do modelo no conjunto de validação: {accuracy_val:.4f}")

## 7. Apresentar a matriz de confusão e a acurácia para o modelo gerado

In [ ]:
# Fazer predições no conjunto de testes
y_pred_test = model.predict(X_test_scaled)

# Calcular a acurácia no conjunto de testes
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Acurácia do modelo no conjunto de testes: {accuracy_test:.4f}")

# Gerar a matriz de confusão
cm = confusion_matrix(y_test, y_pred_test)
print("\nMatriz de Confusão para o conjunto de testes:")
print(cm)

# Visualizar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

## 8. Apresentar o resultado de uma predição do modelo implantado

In [ ]:
# Selecionar uma amostra para predição (primeira amostra do conjunto de testes)
sample_to_predict = X_test_scaled.iloc[0].values.reshape(1, -1)
true_label = y_test.iloc[0]

print(f"Amostra de entrada para predição: {X_test.iloc[0].values}")
print(f"Label verdadeiro para a amostra: {true_label}")

# Realizar a predição
predicted_label = model.predict(sample_to_predict)[0]
print(f"Label predito pelo modelo: {predicted_label}")